# plot_cloud_mask.ipynb

Code to plot time series of cloud mask from model and measurements.

In [ ]:
import xarray as xr
import pandas as pn
import numpy as np
import numpy.ma as ma
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import capricorn_figure_props as fprops
import capricorn_functions as cfn
import capricorn_constants as const
from simulations import simulations
from glob import glob
from importlib import reload
reload(cfn)
reload(const)


## Load CAP measurements from file

In [ ]:
# Cloud mask radar and lidar
radar_files = glob('data/CAPRICORN/LIDAR-RADAR*')
fn_fmt = 'data/CAPRICORN/LIDAR-RADAR_merged.%Y%m%d.nc'
jday_const = 1721425.5  # number of days from 1 Jan 4712 BC to 1 Jan 1 AD
jday_ref_time = dt.datetime(1,1,1)
cld_mask_da = []
for file in radar_files:
    print(file)
    ds = xr.open_dataset(file, decode_times=False)['cld_mask']
    time_name = 'hour'
    ref_time = dt.datetime.strptime(file, fn_fmt)
    time = np.array([ref_time + dt.timedelta(seconds=X*3600)
                     for X in ds[time_name].values])
    cld_mask_da.append(xr.DataArray(
        data = ds.values,
        coords = {
            'height': ds['height'],
            'time': time,
        }
    ))
cap_cld_mask = xr.concat(cld_mask_da, dim='time')

cap_cld_mask_1hr = cap_cld_mask.resample(time="1h").reduce(ma.mean)


## Load model output

In [ ]:
model_twc = []
model_cf = []
model_cfice = []
i_fig_sim = [0,1]
for sim in simulations[i_fig_sim]:
    print(sim.code)
    # load air density in kg m-3, to convert mass mixing ratio to concentrations
    rho = cfn.load_air_density(sim.code, grid=False)
    
    liquid_mass = xr.open_dataarray(f'data/model/{sim.code}_m01s00i254_CAP.nc')
    lwc = liquid_mass*rho.air_density
    ice_mass = xr.open_dataarray(f'data/model/{sim.code}_m01s00i012_CAP.nc')
    iwc = ice_mass*rho.air_density
    model_twc.append(lwc+iwc)

    # load cloud fraction
    cfliq = xr.open_dataarray(f'data/model/{sim.code}_m01s00i267_CAP.nc')
    model_cf.append(cfliq)
    cfice = xr.open_dataarray(f'data/model/{sim.code}_m01s00i268_CAP.nc')
    model_cfice.append(cfice)


In [ ]:
twc_low_lim = 1000 # m
twc_hi_lim = 4000
twc_low_thresh = 1e-3 # g m-3
twc_hi_thresh = 1e-4
model_cld_mask = []
for s,sim in enumerate(simulations[i_fig_sim]):
    # calculate cloud mask based on TWC
    model_z = model_twc[s].TH_1_60_eta_theta*const.ztop
    twc_thresh = np.interp(model_z, [twc_low_lim,twc_hi_lim],[twc_low_thresh,twc_hi_thresh])
    twc_thresh = np.expand_dims(twc_thresh, axis=(0))
    twc_mask = 1000*model_twc[s]>= twc_thresh
    model_cld_mask.append(twc_mask)


## Plot cloud mask comparison

In [ ]:
cmap = matplotlib.colormaps['Blues_r']
for s,sim in enumerate(simulations[i_fig_sim]):
    # average of obs cloud mask
    cld_mask_subset_1hr = cld_mask_subset[s].resample(time="1h").reduce(ma.mean)
    # plot
    fig = plt.figure(figsize=(20*fprops.in_cm,12*fprops.in_cm), dpi=fprops.dpi)
    ax1, ax2 = fig.subplots(2, 1)
    sim = simulations[s]
    model_z = 1e-3*model_cld_mask[s].TH_1_60_eta_theta*const.ztop
    (cld_mask_subset_1hr>0).plot(ax=ax1, cmap=cmap, add_colorbar=False, add_labels=False)
    ax2.pcolormesh(model_cld_mask[s].T1HR.values,
                   model_z, model_cld_mask[s].T, cmap=cmap)
    ax1.set_xlim(ax2.get_xlim())
    ax1.set_xticklabels([])
    ax1.set_ylim(0,10)
    ax2.set_ylim(0,10)
    ax1.set_title('(a)', loc='left', fontsize=fprops.ax_fs)
    ax1.set_title('CAPRICORN-2 cloud mask product', fontsize=fprops.ax_fs)
    ax2.set_title('(b)', loc='left', fontsize=fprops.ax_fs)
    ax2.set_title('Model cloud mask from total water content', fontsize=fprops.ax_fs)
    if sim.code == 'u-cu657':
        fig.suptitle('Ocean', fontsize=fprops.label_fs, y=0.98)
    else:
        fig.suptitle('Coast', fontsize=fprops.label_fs, y=0.98)
    ax2.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax1.tick_params(axis='y', labelsize=fprops.ax_fs)
    ax2.tick_params(axis='y', labelsize=fprops.ax_fs)
    xtick_fmt = '%d-%b'
    ax2.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
    fig.supylabel('Altitude [km]', fontsize=fprops.ax_label_fs, x=0.05)
    filename = f'figures/cloud_masks_{sim.code}.png'
    plt.savefig(filename, bbox_inches='tight', facecolor='white')

    # calculate rough cloud base/top
    icap_cld_top = []
    icap_cld_base = []
    for i in np.arange(len(cld_mask_subset_1hr.time)):
        if ma.any(ma.masked_invalid(cld_mask_subset_1hr.isel(time=i).values)):
            icap_cld_top.append(np.amax(np.where(ma.masked_invalid(cld_mask_subset_1hr.isel(time=i)>0))[0]))
            icap_cld_base.append(np.amin(np.where(ma.masked_invalid(cld_mask_subset_1hr.isel(time=i)>0))[0]))
        else:
            icap_cld_top.append(0)
            icap_cld_base.append(0)
    cap_cld_top = cld_mask_subset_1hr.height[icap_cld_top]
    cap_cld_base = cld_mask_subset_1hr.height[icap_cld_base]
    cap_cld_top = ma.masked_where(ma.any(ma.masked_invalid(cld_mask_subset_1hr.values), axis=0)==0, cap_cld_top)
    cap_cld_base = ma.masked_where(ma.any(ma.masked_invalid(cld_mask_subset_1hr.values), axis=0)==0, cap_cld_base)

    k_cld_top_twc = []
    k_cld_base_twc = []
    for i in np.arange(len(model_twc[s].T1HR)):
        if np.any(model_cld_mask[s].isel(T1HR=i).values):
            k_cld_top_twc.append(np.amax(np.where(model_cld_mask[s].isel(T1HR=i)>0)[0]))
            k_cld_base_twc.append(np.amin(np.where(model_cld_mask[s].isel(T1HR=i)>0)[0]))
        else:
            k_cld_top_twc.append(0)
            k_cld_base_twc.append(0)
    cld_top_twc = model_z[k_cld_top_twc]
    cld_base_twc = model_z[k_cld_base_twc]
    cld_top_twc = ma.masked_where(ma.any(model_cld_mask[s], axis=1)==0, cld_top_twc)
    cld_base_twc = ma.masked_where(ma.any(model_cld_mask[s], axis=1)==0, cld_base_twc)

    fig = plt.figure(figsize=(20*fprops.in_cm,5*fprops.in_cm), dpi=fprops.dpi)
    plt.fill_between(x=cld_mask_subset_1hr.time, y1=cap_cld_base, y2=cap_cld_top, color='k', alpha=0.5, edgecolor='None')
    plt.fill_between(x=model_cld_mask[s].T1HR.values, y1=cld_base_twc, y2=cld_top_twc, color=sim.colour, alpha=0.5, edgecolor='None')
    plt.xlim(ax1.get_xlim())
    plt.ylim(bottom=0)
    plt.ylabel('Height [km]')
